# Notebook objectives
* Define machine learning
* Discuss estimation equations and cost/loss functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor

%matplotlib inline

# What is machine learning according to a Google search?  


<br>
<font color="red" size="4">**Does this image increase your understanding of ML?**</font>

<img src="images/ml_brain.jpg" align="left"></img> 

# Machine learning is estimating and minimizing error

<img src="images/lin_reg.jpg" width=700 align="right"></img>

**x** = independent variable   
**y** = dependent or target variable  
**^** used to denote an estimate  
**-** used to denote the mean

### Possibly familiar format
* Estimation equation:  $\hat{y} = mx + b$
* Scoring:  $(y - \hat{y})^2$ 

### Machine learning format 
* Estimation equation:  $\hat{y} = \beta_0 + \beta_1 x_1...$
* Cost or loss function: $(y - \hat{y})^2$

<br>
<font color="red" size="4">**Why square the difference?**</font> 

# Some common syntax

* Scalar is a number
* Vector is a collection of scalars that is 1-dimensional (a sequence of scalars)
    * Typically represented with a lowercase letter such as y
* Matix is a collection of vectors that generally 2-dimensional (a table of data)
    * Typically represented with an uppercase letter such as X
    * The rows or columns of the matrix are vectors  

<br>
<font color="red" size="4">**Which one of these terms generally describes the target variable?**</font>
<br>
<font color="red" size="4">**Which one of these terms generally describes the data for your model?**</font>
<br>
<br>
<img src="images/scalar-vector-matrix.svg" align="left"></img>

# Types of machine learning

<img src="images/ml_types.jpg" align="right"></img>

### Supervised learning (y is known)
#### Regression (y is continuous)
* Linear models (linear, polynomial, Poisson regressions, etc)
* Time series forecasting (autoregression)
* Tree models (decision trees, random forests, boosted trees)  

#### Classification (y represents a class)
* K-nearest neighbors
* Logistic regression
* Support vector machines
* Tree models (decision trees, random forests, boosted trees)
* Neural networks (deep learning)

### Unsupervised learning (y is unknown)
#### Clustering (creates groups and assigns y)
* K-means (groups by distance from centroids)
* Hierarchical (groups based on similarity)
* DBSCAN (groups based upon density)

<br>
<font color="red" size="4">**What other machine learning algorithsms missing from the list?**</font>  
<br>
<font color="red" size="4">**What are some business that make their money off regression? How about classification?**</font>

# Load data
Original data obtained from obtained from [Seattle Real Time Fire 911 Calls](https://data.seattle.gov/Public-Safety/Seattle-Real-Time-Fire-911-Calls/kzjm-xkqj). The [Seattle Open Data Portal](https://data.seattle.gov/browse?limitTo=datasets) has a bunch of fun real (and sometimes messy) data to play with.

Features were created and data was aggegrated to generate the dataset found in the data directory. This is the same data that powers my project [Rapid-Rescue](http://rapid-rescue.com/) that uses Poisson regression and K-means clustering to estimate number of medical emergencies in each area of Seattle and optimal placements for emergency response vehicles given historical data.

<br>
<font color="red" size="4">**What are some interesting pieces of data to study here at Nordstrom?**</font>

In [ ]:
# load data, check for nulls, ensure it's properly cast, and create X and y
df = pd.read_csv('../data/model_data.csv', index_col=0)
data_cols = [col for col in df.columns if col != 'freq']
X = df[data_cols]
y = df.freq
df.info()

In [ ]:
# look at first few rows
df.head(10)

In [ ]:
df.describe()

# Split data into training and tests sets

* The training set is used to train the model
* The test data set is used to validate the model and explore its generalizability

<br>
<font color="red" size="4">**Why is it important to split the data?**</font>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=97)

depths = xrange(1,36)
train_scores, test_scores = [], []

for depth in depths:
    DTR = DecisionTreeRegressor(max_depth=depth)
    DTR.fit(X_train, y_train)
    train_scores.append(-DTR.score(X_train, y_train))
    test_scores.append(-DTR.score(X_test, y_test))    

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(depths, train_scores)
plt.ylabel('Error')
plt.xlabel('Model Complexity')
plt.title('Decision Tree Regressor Train Error');

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(depths, train_scores, label='train')
plt.scatter(depths, test_scores, label='test')
plt.ylabel('Error')
plt.xlabel('Model Complexity')
plt.title('Decision Tree Regressor Train and Test Error')
plt.legend();

# Bias/variance trade off and under/overfitting

* Bias refers to the error introduced by approximating a complex real-world problem with a simpler model (rigidity and relationship assumptions)
* Variance describes the degree to which the estimation equation will vary if trained on a different dataset (flexibility)

If the model has overly high bias, it will be too inflexible to effectively approximate the data. This type of model is **underfit** (see below left). If the model has overly high variance, it will not be generalizable to other data. When the model fits the training data, but is not generalizable to other data, it is said to be **overfit** (see below right).

[Matt Drury](https://github.com/madrury) created a fantastic [visualization tool](http://madrury.github.io/smoothers/) for bias and variance.

<br>
<font color="red" size="4">**How can we determine if a model is underfit? **</font>  
<br>
<font color="red" size="4">**How can we determine if a model is overfit?**</font>


<img src="images/bias_variance.png" align="left"></img>

# What are hyperparameters?

* Parameters of the model that are not directly "learned" from the data
* Generally set before the model is run
* Optimized through trial and error
* Often controls bias/variance trade off for model

<br>
<font color="red" size="4">**Wouldn't it be great if there was an easy way to test hyperparameters?**</font>

# Grid searching and cross-validation

* K-fold cross validation breaks training data into K training sets and K test sets
* Tools in Python and R for automating the process of use cross validation to test combinations of hyperparemeters 

<br>
<font color="red" size="4">**Why is it desirable to create more training sets?**</font>  
<br>
<font color="red" size="4">**What problems might this help us spot?**</font>

<img src="images/kfold.gif" align="left"></img>

In [ ]:
# instantiate model and set hyperparaters to test, and instantiate GridSearchCV
DTR = DecisionTreeRegressor()
param_grid = {'max_depth': range(1, 35)}
GSCV = GridSearchCV(estimator=DTR,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=3,
                    return_train_score=True)

GSCV.fit(X_train, y_train)

In [ ]:
sorted(GSCV.cv_results_.keys())

In [ ]:
plt.figure(figsize=(12, 6))
x_vals = range(1, 35)
plt.scatter(x_vals, -GSCV.cv_results_['split0_test_score'], label='split0_test_score')
plt.scatter(x_vals, -GSCV.cv_results_['split1_test_score'], label='split1_test_score')
plt.scatter(x_vals, -GSCV.cv_results_['split2_test_score'], label='split2_test_score')
plt.scatter(x_vals, -GSCV.cv_results_['split0_train_score'], label='split0_train_score')
plt.scatter(x_vals, -GSCV.cv_results_['split1_train_score'], label='split1_train_score')
plt.scatter(x_vals, -GSCV.cv_results_['split2_train_score'], label='split2_train_score')
plt.ylabel('Error')
plt.xlabel('Model Complexity')
plt.title('Decision Tree Regressor Train and Test Error')
plt.legend();

# Let's see what happens with a data issue

In [ ]:
# make some random data and add it to our training data
y_bad = np.random.randint(0, 100, 3000)
X_bad = np.random.rand(3000, 21)
y_train_bad = np.append(y_train, y_bad)
X_train_bad = np.append(X_train, X_bad, axis=0)

In [ ]:
GSCV.fit(X_train_bad, y_train_bad)

In [ ]:
plt.figure(figsize=(12, 6))
x_vals = range(1, 35)
plt.scatter(x_vals, -GSCV.cv_results_['split0_test_score'], label='split0_test_score')
plt.scatter(x_vals, -GSCV.cv_results_['split1_test_score'], label='split1_test_score')
plt.scatter(x_vals, -GSCV.cv_results_['split2_test_score'], label='split2_test_score')
plt.scatter(x_vals, -GSCV.cv_results_['split0_train_score'], label='split0_train_score')
plt.scatter(x_vals, -GSCV.cv_results_['split1_train_score'], label='split1_train_score')
plt.scatter(x_vals, -GSCV.cv_results_['split2_train_score'], label='split2_train_score')
plt.ylabel('Error')
plt.xlabel('Model Complexity')
plt.title('Decision Tree Regressor Train and Test Error')
plt.legend();